[View in Colaboratory](https://colab.research.google.com/github/shubhamitradas/Toxicity-Challenge/blob/master/Elmo_keras.ipynb)

In [1]:
# Import our dependencies
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re
from keras import backend as K
import keras.layers as layers
from keras.models import Model
import numpy as np

# Initialize session
sess = tf.Session()
K.set_session(sess)

Using TensorFlow backend.


In [2]:

# Now instantiate the elmo model# Now i 
elmo_model = hub.Module("https://tfhub.dev/google/elmo/1", trainable=True)
sess.run(tf.global_variables_initializer())
sess.run(tf.tables_initializer())

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/elmo/1'.
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/elmo/1'.
INFO:tensorflow:Initialize variable module/aggregation/scaling:0 from checkpoint b'/tmp/tfhub_modules/0eadd30f548cbfd3065119697ded6bb381668285/variables/variables' with aggregation/scaling
INFO:tensorflow:Initialize variable module/aggregation/weights:0 from checkpoint b'/tmp/tfhub_modules/0eadd30f548cbfd3065119697ded6bb381668285/variables/variables' with aggregation/weights
INFO:tensorflow:Initialize variable module/bilm/CNN/W_cnn_0:0 from checkpoint b'/tmp/tfhub_modules/0eadd30f548cbfd3065119697ded6bb381668285/variables/variables' with bilm/CNN/W_cnn_0
INFO:tensorflow:Initialize variable module/bilm/CNN/W_cnn_1:0 from checkpoint b'/tmp/tfhub_modules/0eadd30f548cbfd3065119697ded6bb381668285/variables/variables' with bilm/CNN/W_cnn_1
INFO:tensorflow:Initialize variable module/b

INFO:tensorflow:Initialize variable module/bilm/RNN_0/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/kernel:0 from checkpoint b'/tmp/tfhub_modules/0eadd30f548cbfd3065119697ded6bb381668285/variables/variables' with bilm/RNN_0/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/kernel
INFO:tensorflow:Initialize variable module/bilm/RNN_0/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel:0 from checkpoint b'/tmp/tfhub_modules/0eadd30f548cbfd3065119697ded6bb381668285/variables/variables' with bilm/RNN_0/RNN/MultiRNNCell/Cell1/rnn/lstm_cell/projection/kernel
INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/bias:0 from checkpoint b'/tmp/tfhub_modules/0eadd30f548cbfd3065119697ded6bb381668285/variables/variables' with bilm/RNN_1/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/bias
INFO:tensorflow:Initialize variable module/bilm/RNN_1/RNN/MultiRNNCell/Cell0/rnn/lstm_cell/kernel:0 from checkpoint b'/tmp/tfhub_modules/0eadd30f548cbfd3065119697ded6bb381668285/variables/variables' with b

In [5]:
train_df.head(2)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0


In [10]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


########################################
## process texts in datasets
########################################
print('Processing text dataset')
train_df = pd.read_csv('train_toxic.csv',nrows=10)
test_df = pd.read_csv('test_toxic.csv')



MAX_SEQUENCE_LENGTH = 100




# Create datasets (Only take up to 150 words for memory)# Create 
train_text = train_df['comment_text'].tolist()
train_text = [' '.join(t.split()[0:150]) for t in train_text]
train_text = np.array(train_text, dtype=object)[:, np.newaxis]

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_label = train_df[list_classes].values

data = train_text
y = train_label



VALIDATION_SPLIT = 0.1

########################################
## sample train/validation data
########################################
np.random.seed(1234)
perm = np.random.permutation(len(data))
idx_train = perm[:int(len(data)*(1-VALIDATION_SPLIT))]
idx_val = perm[int(len(data)*(1-VALIDATION_SPLIT)):]

data_train=data[idx_train]
labels_train=y[idx_train]
print(data_train.shape,labels_train.shape)

data_val=data[idx_val]
labels_val=y[idx_val]

print(data_val.shape,labels_val.shape)

Processing text dataset
(9, 1) (9, 6)
(1, 1) (1, 6)


In [4]:
train_label

array([[0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [12]:
# Build our model

# We create a function to integrate the tensorflow model with a Keras model
# This requires explicitly casting the tensor to a string, because of a Keras quirk
def ElmoEmbedding(x):
    return elmo_model(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]
 
input_text = layers.Input(shape=(1,), dtype=tf.string)
embedding = layers.Lambda(ElmoEmbedding, output_shape=(1024,))(input_text)
dense = layers.Dense(256, activation='relu')(embedding)
pred = layers.Dense(6, activation='softmax')(dense)

model = Model(inputs=[input_text], outputs=pred)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1)                 0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 1024)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 1542      
Total params: 263,942
Trainable params: 263,942
Non-trainable params: 0
_________________________________________________________________


In [13]:
hist = model.fit(data_train, labels_train, \
        validation_data=(data_val, labels_val), \
        epochs=5, batch_size=64,verbose=2, shuffle=True)



Train on 9 samples, validate on 1 samples
Epoch 1/5


InvalidArgumentError: ignored

In [0]:
test_text = test_df['comment_text'].tolist()
test_text = [' '.join(t.split()[0:150]) for t in test_text]
test_text = np.array(test_text, dtype=object)[:, np.newaxis]

In [0]:
y_test = model.predict([test_text], batch_size=64, verbose=2)

sample_submission = pd.read_csv("sample_toxic.csv")
sample_submission[list_classes] = y_test

sample_submission.to_csv('submission.csv', index=False)

In [0]:
!gzip submission.csv
from google.colab import files


files.download('submission.csv.gz')

In [10]:
train_text

array([["Explanation Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"],
       ["D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)"],
       ["Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info."],
       ...,
       ['Spitzer Umm, theres no actual article for prostitution ring. - Crunch Captain.'],
       ['And it looks like it was actually you who put on the speedy to have the first version deleted now that I look at it.'],
       ['" And ... I really don\'t think you understand. I came here and my idea was bad right away. What kind of communi